In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.metrics import MeanIoU



In [14]:
image_dir=r'dataaset\images'
mask_dir=r'dataaset\masks'

image_size = (224, 224)
num_classes = 3
batch_size=2
epochs=10

image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir)])
mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir)])

In [15]:
image_paths = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir)])
mask_paths = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir)])

assert len(image_paths) == len(mask_paths), "Mismatch in number of images and masks"
print("Total pairs:", len(image_paths))

Total pairs: 109


In [16]:
def load_and_preprocess(image_path, mask_path):
    # Load and resize image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.cast(image, tf.float32) / 255.0

    # Load and resize mask
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)  # Force 1 channel
    mask = tf.image.resize(mask,image_size, method='nearest')
    mask = tf.squeeze(mask, axis=-1)              # Remove channel
    mask = tf.cast(mask, tf.int32)                # For sparse_categorical_crossentropy

    return image, mask


In [17]:
dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths))
dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.cache().shuffle(buffer_size=100)
train_ds = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [18]:
for batch_images, batch_masks in train_ds.take(1):
    print("🧪 Image batch shape:", batch_images.shape)
    print("🧪 Mask batch shape:", batch_masks.shape)

🧪 Image batch shape: (2, 224, 224, 3)
🧪 Mask batch shape: (2, 224, 224)


In [19]:
# def generate_fake_dataset(batch_size=2, img_size=(256, 256), num_classes=3):
#     def gen():
#         for _ in range(100):
#             img = tf.random.uniform((img_size[0], img_size[1], 3), dtype=tf.float32)
#             mask = tf.random.uniform((img_size[0], img_size[1]), maxval=num_classes, dtype=tf.int32)
#             yield img, mask

#     dataset = tf.data.Dataset.from_generator(
#         gen,
#         output_signature=(
#             tf.TensorSpec(shape=(256, 256, 3), dtype=tf.float32),
#             tf.TensorSpec(shape=(256, 256), dtype=tf.int32)
#         )
#     )
#     return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# fake_train = generate_fake_dataset()


In [26]:
def upsample_block(filters,kernel_size):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2DTranspose(filters,kernel_size,strides=2,padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.ReLU()
    ])

In [27]:
def build_unet_mobilenetv2(input_shape, num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')

    # Use intermediate layers for skip connections
    layer_names = [
        'block_1_expand_relu',   # 112x112
        'block_3_expand_relu',   # 56x56
        'block_6_expand_relu',   # 28x28
        'block_13_expand_relu',  # 14x14
        'out_relu'               # 7x7
    ]
    layers = [base_model.get_layer(name).output for name in layer_names]

    down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)
    down_stack.trainable = True  # Enable fine-tuning

    inputs = tf.keras.Input(shape=input_shape)
    skips = down_stack(inputs)
    x = skips[-1]  # Start from 'out_relu' (7x7)

    # Decoder: upsampling and concatenation with skip connections
    x = upsample_block(512, 3)(x)           # 14x14
    x = tf.keras.layers.Concatenate()([x, skips[-2]])

    x = upsample_block(256, 3)(x)           # 28x28
    x = tf.keras.layers.Concatenate()([x, skips[-3]])

    x = upsample_block(128, 3)(x)           # 56x56
    x = tf.keras.layers.Concatenate()([x, skips[-4]])

    x = upsample_block(64, 3)(x)            # 112x112
    x = tf.keras.layers.Concatenate()([x, skips[-5]])

    x = upsample_block(32, 3)(x)            # 224x224

    # Final segmentation head
    outputs = tf.keras.layers.Conv2D(num_classes, 1, activation='softmax')(x)

    return tf.keras.Model(inputs, outputs)

In [32]:
model = build_unet_mobilenetv2(image_size + (3,), num_classes)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 model_6 (Functional)        [(None, 112, 112, 96),       2257984   ['input_8[0][0]']             
                              (None, 56, 56, 144),                                                
                              (None, 28, 28, 192),                                                
                              (None, 14, 14, 576),                                                
                              (None, 7, 7, 1280)]                                                 
                                                                                            

In [ ]:
history = model.fit(train_ds, epochs=epochs)


Epoch 1/10


55/55 [==============================] - 66s 934ms/step - loss: 0.6854 - accuracy: 0.7046
Epoch 2/10
55/55 [==============================] - 47s 860ms/step - loss: 0.4104 - accuracy: 0.8310
Epoch 3/10
49/55 [=========================>....] - ETA: 5s - loss: 0.3462 - accuracy: 0.8557